In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout
from keras_radam import RAdam
from keras.optimizers import RMSprop
import pandas
import ast


Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
train_set = ".\\dataset\\train"
validation_set = ".\\dataset\\val"

In [5]:
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [6]:
train_df = pandas.read_csv(".\\dataset\\train\\output_final.csv")
valid_df = pandas.read_csv(".\\dataset\\val\\output_final.csv")

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=20, 
    brightness_range=[0.4,1.0], 
    horizontal_flip=True, 
    rescale=1./255, 
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='.\\dataset\\train',
        x_col="filename",
        y_col=["x_start","y_start","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(300, 300),
        batch_size=128)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='.\\dataset\\val',
        x_col="filename",
        y_col=["x_start","y_start","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(300, 300),
        batch_size=128)

Found 3606 validated image filenames.
Found 801 validated image filenames.


In [8]:
step_size_train = train_generator.n/train_generator.batch_size
step_size_valid = valid_generator.n/valid_generator.batch_size

In [9]:
vgg_model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg', input_shape=(300, 300, 3))

In [10]:
vgg_model.trainable = True

In [11]:
vgg_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
________________________________________________________________________________

In [12]:
for l in vgg_model.layers:
    #if "conv" not in l.name:
        l.trainable = False

In [13]:
vgg_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
________________________________________________________________________________

Total params: 54,336,736
Trainable params: 0
Non-trainable params: 54,336,736
__________________________________________________________________________________________________


In [14]:
model = Sequential()
model.add(vgg_model)

model.add(Dense(1536,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1536,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1536,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4,activation='relu'))

In [15]:
# For a mean squared error regression problem

#model.compile(optimizer=RAdam(total_steps=5000, warmup_proportion=0.1, min_lr=1e-5), loss='mse', metrics=['accuracy'])
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='mse', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dense_1 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1536)             

In [17]:
model.evaluate_generator(valid_generator, steps=step_size_valid)

[116937.46875, 0.006242197472602129]

In [18]:
history = model.fit_generator(generator=train_generator, epochs=25, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=2)

Epoch 1/25
 - 88s - loss: 52376.6658 - accuracy: 0.6578 - val_loss: 39259.9648 - val_accuracy: 0.6941
Epoch 2/25
 - 63s - loss: 29015.2775 - accuracy: 0.7238 - val_loss: 57350.7188 - val_accuracy: 0.6941
Epoch 3/25
 - 62s - loss: 28168.3574 - accuracy: 0.7238 - val_loss: 45883.4141 - val_accuracy: 0.6941
Epoch 4/25
 - 63s - loss: 27410.2099 - accuracy: 0.7238 - val_loss: 48124.3125 - val_accuracy: 0.6941
Epoch 5/25
 - 63s - loss: 25678.2581 - accuracy: 0.7238 - val_loss: 43434.0781 - val_accuracy: 0.6941
Epoch 6/25
 - 63s - loss: 23837.3656 - accuracy: 0.7238 - val_loss: 48748.9844 - val_accuracy: 0.6941
Epoch 7/25
 - 63s - loss: 20845.3571 - accuracy: 0.7571 - val_loss: 54909.9375 - val_accuracy: 0.6866
Epoch 8/25
 - 63s - loss: 18864.1821 - accuracy: 0.7895 - val_loss: 51558.0625 - val_accuracy: 0.5531
Epoch 9/25
 - 62s - loss: 18091.4150 - accuracy: 0.7945 - val_loss: 48616.4414 - val_accuracy: 0.5368
Epoch 10/25
 - 62s - loss: 17966.0113 - accuracy: 0.7912 - val_loss: 63838.3555 - 

KeyboardInterrupt: 

In [ ]:
print(history.history)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,80000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)


# In[ ]:


import pandas as pd
dict_data = history.history
df = pd.DataFrame.from_dict(dict_data, dtype=float)
csv_file = "history_resnet.csv"
df.to_csv(csv_file)


# In[ ]:


model_json = model.to_json()
with open("resnet50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("resnet50.h5")
print("Saved model to disk")